In [ ]:
import pandas as pd
import glob
import numpy as np

from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
import tensorflow as tf

import time


import matplotlib.pyplot as plt

from sklearn.model_selection import KFold

# PREPARACION DE LOS DATOS:

In [ ]:
csv = glob.glob("/Users/usuario/Desktop/CALLS/201*_FINAL_CALLS/C*_FINAL.csv")


x_data = [ 0 ]
y_data = [ 0 ]

for j in range(len(csv)):
    
    data = pd.read_csv(csv[j])
    nom_fec = csv[j][-17:-10]
  
    #Reducimos todos los dias a solo 5 maturity para tener la misma dimension en todos:
    #buscaremos cuantas maturitys tiene cada dia: 
    #- Si coincide con 5 no hacemos nada
    # - Si es superior eliminaremos la menos o las dos menores segun tenga 6 o 7.
    a = data.Diferencia.unique()
    a.sort()  
    if len(data.Diferencia.unique()) == 7:
        index_names = data[ data['Diferencia'] == a[5] ].index
        data.drop(index_names , inplace = True)  
        index_names2 = data[ data['Diferencia'] == a[6] ].index
        data.drop(index_names2 , inplace = True)  
    elif len(data.Diferencia.unique()) == 8:
        index_names = data[ data['Diferencia'] == a[5] ].index
        data.drop(index_names , inplace = True)  
        index_names2 = data[ data['Diferencia'] == a[6] ].index
        data.drop(index_names2 , inplace = True)
        index_names3 = data[ data['Diferencia'] == a[7] ].index
        data.drop(index_names3 , inplace = True) 
    elif len(data.Diferencia.unique()) == 9:
        index_names = data[ data['Diferencia'] == a[5] ].index
        data.drop(index_names , inplace = True)  
        index_names2 = data[ data['Diferencia'] == a[6] ].index
        data.drop(index_names2 , inplace = True)
        index_names3 = data[ data['Diferencia'] == a[7] ].index
        data.drop(index_names3 , inplace = True) 
        index_names4 = data[ data['Diferencia'] == a[8] ].index
        data.drop(index_names4 , inplace = True) 
    else:
        data = data
        
    #print(len(data.Diferencia.unique()))   
    #Buscamos el valor de moneyness=1 y la menor maturity para normalizar:   
    norm = data.iloc[:,2:]
    norm = norm.sort_values('Diferencia')
    norm.reset_index(inplace = True)
    norm = norm.iloc[:,1:]
    
    i = 0
    for i in range(len(norm[['moneyness']])):
        #print(i)
        #print(norm.iloc[i,6:9])
        if norm.moneyness.iloc[i] >= 1.00000000 and norm.moneyness.iloc[i] <= 1.010000000 :
            break
        elif norm.moneyness.iloc[i] >= 1.01000000 and norm.moneyness.iloc[i] <= 1.020000000 :
            break
        else:
            i += 1
     
    #Dividimos los valores por el valor elegido
    vola_norm = norm['new_vola'] / norm.new_vola.iloc[i]
    precio_norm = norm['Precio'] / norm.Precio.iloc[i] 
    
    norm.loc[:, 'vola_norm'] = vola_norm
    norm.loc[:, 'precio_norm'] = precio_norm
    final = norm
    
    #Creamos la matriz de valores que sirve para representar la superficie de volatilidad:
    
    a = final[['Diferencia','moneyness','vola_norm', 'precio_norm' , 'strike_norm' ]]
    b = a.drop_duplicates()

    matriz_vola = b.set_index(['Diferencia', 'moneyness'])['vola_norm'].unstack(level=-1)
   
    
    #Nos quedamos con matrices 5x11:
    #Para ello buscamos la posicion del valor 1.0000 en la menor maturity 
    # y cogemos (-6,+6) posiciones a partir de ese columna:
    k=0
    for k in range(len(matriz_vola.iloc[0,  : ])):
        if matriz_vola.iloc[0,k] == 1.000000 and norm.Diferencia.iloc[k] == min(norm.Diferencia.unique()) :
            break
        else:
            k = k+1
          
    matriz_vola_norm = matriz_vola.iloc[:, k-6 : k+6 ]
     
    matriz_precio = b.set_index(['Diferencia', 'moneyness'])['precio_norm'].unstack(level=-1)
    matriz_precio_norm = matriz_precio.iloc[:, k-6 : k+6 ]

    #Rellenamos si algun hueco queda Nan:
    matriz_vola_norm = matriz_vola_norm.fillna(axis=1, method='bfill')
    matriz_vola_norm = matriz_vola_norm.fillna(axis=1, method='ffill')

    matriz_precio_norm = matriz_precio_norm.fillna(axis=1, method='bfill')
    matriz_precio_norm = matriz_precio_norm.fillna(axis=1, method='ffill')
    
    #Creamos los vectores unidmensionles a partir de las matrices anteriores:
    p_data = matriz_precio_norm.stack()
    v_data = matriz_vola_norm.stack()
    
    if len(data.Diferencia.unique()) == 5:
        #Añadimos la ultima fila de la matriz repetida en caso de ser solo de 5 maturities:
        #Para la del precio:
        e = p_data[-12:]
        p_data_mod = np.concatenate([p_data,e]) 
        p_data = p_data_mod
        #Para la volatilidad:
        e = v_data[-12:]
        v_data_mod = np.concatenate([v_data,e]) 
        v_data = v_data_mod
        print('Hemos añadido una')
    else:
        p_data = p_data.values
        v_data = v_data.values
        
    #Crearemos un array unimensionl y de len() igual a (nº años) x (dias del año) x (len(dia))
    #Despues se redimensionara para que sea por filas los datos de cada dia: 
    # shape[ (nº años) x (dias del año) , 72 ]
    x_data = np.concatenate([x_data , p_data])
    y_data = np.concatenate([y_data , v_data])
    
    
   
    nom_fec = csv[j][-17:-10]
    

### Eliminamos los valores de 0 que introducimos al princpio que no son datos.

In [ ]:
a_x = np.delete(x_data, (0,0))
a_y = np.delete(y_data, (0,0))

### Guardamos los datos como una lista sin dimnsiones que será utilizada en todos los modelos con las modificaciones de dimensiones necesarias.

In [ ]:
np.savetxt('datos_x_2.txt' , a_x , delimiter=',')
np.savetxt('datos_y_2.txt' , a_y , delimiter=',')